In [ ]:
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library
import matplotlib as mpl
import matplotlib.pyplot as plt
import requests # library to handle requests
import random # library for random number generation

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# !conda install -c conda-forge folium=0.5.0 --yes
import folium

#! conda install -c conda-forge ipywidgets --yes
import ipywidgets as widgets

print('Folium installed and imported!')

In [ ]:
# @hidden_cell
CLIENT_ID = ' ' # your Foursquare ID
CLIENT_SECRET = ' ' # your Foursquare Secret
VERSION = '20200501'
LIMIT = 100 # max only 50 result 

radius = 10000

In [ ]:
delhi_district_file_path='./delhi_all.csv'
delhi_district=pd.read_csv(delhi_district_file_path)

delhi_population_path='./delhi9_population.csv'
delhi_population=pd.read_csv(delhi_population_path)

delhi_geo_path='./Delhi.geojson'
import json
#delhi_geo = json.loads(delhi_geo_path)
with open(delhi_geo_path, 'r') as j:
     delhi_geo = json.loads(j.read())
        
#delhi_geo=pd.read_csv(delhi_geo_path, error_bad_lines=False)

In [ ]:
 delhi_district.place_name.unique()
# delhi_population.head(3)
# delhi_geo.columns
# print(delhi_geo)

In [ ]:
from IPython.display import clear_output
caption=widgets.Label(value='Not selected yet')
ALL='ALL'
def unique_sorted_values_ALL(array):
    unique=array.unique().tolist()
    unique.sort()
    unique.insert(0,ALL)
    return unique
x='JAMES'
def onchange(d):
    #clear_output(wait=False)
    #print ('%s' % d['new'] )
    pass
    #print('Selection changed event fired.')

district=widgets.Dropdown(
    options=delhi_district.place_name,
    description='Places :'
)
widgets.link((district,'value'),(caption,'value')) 
district.observe(onchange)
display(district, caption) 

In [ ]:
vars(district); 
state=district.value
selectedRow=delhi_district.loc[delhi_district['place_name']==district.value, :]

vars(selectedRow);

In [ ]:
geolocator = Nominatim(user_agent="foursquare_agent", format_string="%s, New delhi,IN")
# location = geolocator.geocode(district.value)
# location = geolocator.reverse("28.509669, 77.376294")
# latitude = location.latitude
# longitude = location.longitude

latitude = selectedRow.loc[district.index,'latitude']
longitude =selectedRow.loc[district.index,'longitude']
location = geolocator.reverse("{}, {}".format(latitude, longitude))
 
# location.raw

In [ ]:
# Delhi latitude and longitude values
delhi_map=folium.Map(location=[latitude, longitude], zoom_start=10.47)

# Add the color for the chloropleth:
delhi_map.choropleth(
 geo_data=delhi_geo,
 name='choropleth',
 data=delhi_population,
 columns=['District', 'Percentage'],
 key_on='feature.properties.Dist_Name',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Population (%)'
)
# folium.LayerControl().add_to(delhi_map)


# I can add marker one by one on the map
for i in range(0,len(delhi_district)):
    folium.Marker([delhi_district.iloc[i]['latitude'], delhi_district.iloc[i]['longitude']], popup=folium.Popup(delhi_district.iloc[i]['place_name'], parse_html=True, max_width=100) ).add_to(delhi_map)

delhi_map

In [ ]:
search_query = 'market'
url = 'https://api.foursquare.com/v2/search/recommendations?client_id={}&client_secret={}&ll={},{}&v={}&query={}&limit={}&state={}&radius={}&sortByDistance=True'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, LIMIT, state, radius)
url
# search/recommendations

In [ ]:
results = requests.get(url).json()
results

In [ ]:
# assign relevant part of JSON to venues
venues = results['response']['group']['results']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe=dataframe[dataframe['venue.location.distance']<=5000]

dataframe.head()
# dataframe['location.address']
